In [1]:
# for collab

import tensorflow as tf

def colab_connect():

    if tf.__version__!='2.13.0':
        %pip install tensorflow==2.13.0
        print('Stopping RUNTIME! Please run again.')
        import os
        os.kill(os.getpid(), 9)

    from google.colab import drive
    drive.mount('/content/drive/',force_remount=True)
    %cd /content/drive/My Drive/LOB/

    %pip install automodinit keras_tuner

try:
    import google.colab
    IS_COLAB = True
    colab_connect()
except:
    IS_COLAB =False


In [2]:
import numpy as np
from tqdm import tqdm
import keras

import data
from utilites import DataClass

seq_len = 100

In [3]:
# Load data
row_data = (
    # data.load_dataset(horizon=4)
    data.load_saved_datas())

data.inspect_datas(row_data)

    Datas:
train     : x= (203799, 40)    | y= (203799,)      
val       : x= (50949, 40)     | y= (50949,)       
test      : x= (139586, 40)    | y= (139586,)      


In [ ]:
# # Save data
# data.save_data(name= 'train',x= x_train,y=y_train)
# data.save_data(name= 'val',x= x_val,y=y_val)
# data.save_data(name= 'test',x= x_test,y=y_test)

# Test model

In [ ]:
from models import m_preln as test_model

In [ ]:
# Datasets
datasets = data.build_datasets(
    datas=row_data,
    batch_size=512,
    seq_len=seq_len,
)
(ds_train, ds_val, ds_test) =\
(datasets['train'], datasets['val'], datasets['test'])
data.inspect_datasets(datasets)

In [ ]:
# Build
model_name = 'base_model_low'
pars = DataClass(test_model.PARAMETRS)
model = test_model.build_model(**pars.Info_expanded)
model.summary()

In [ ]:
# Callbacks

class SaveCallback(keras.callbacks.Callback):

    def on_train_batch_end(self, batch, logs=None):
        if not batch%50:
            !cp -a "/content/drive/MyDrive/LOB/Temp/callbacks" "/content/drive/MyDrive/LOB/Save"


callbacks = [
    keras.callbacks.TensorBoard(
        f"Temp/callbacks/{model_name}",
        histogram_freq=1,
        update_freq=1,
    ),
    tf.keras.callbacks.ModelCheckpoint(
        f"Temp/callbacks/{model_name}/checkPoint",
        monitor="val_loss",
        verbose=0,
        save_best_only=False,
        save_weights_only=True,
        mode="auto",
        save_freq=50,
        options=None,
        initial_value_threshold=None,
    )
]
if IS_COLAB: callbacks +[SaveCallback()]


In [ ]:
# Train
model.fit(
    ds_train,
    epochs=20,
    validation_data=ds_val,
    callbacks=callbacks,
)

# Search

In [4]:
%tensorboard

In [5]:
import datetime

from keras_tuner.tuners import GridSearch
import keras_tuner

import models

In [6]:
# Datasets
datasets = data.build_datasets(
    datas=row_data,
    batch_size=100,
    seq_len=seq_len,
)
(ds_train, ds_val, ds_test) =\
(datasets['train'], datasets['val'], datasets['test'])
data.inspect_datasets(datasets)

    Datasets:
train     : [2037, 100, 40]
val       : [509, 100, 40]
test      : [1395, 100, 40]


In [7]:
# Tuner parametrs
def configure(hp: keras_tuner.HyperParameters):

    class CN_search(DataClass):
        dilation_steps = hp.Int(
            'cn_layers',
            default=4,
            min_value=3,
            max_value=5,
            step=1,
        )

    class AN_search(DataClass):
        attention_heads = 3
        blocks = 2
        share_weights = hp.Boolean(
            'share_weights',
            default=True,
        )

    class FF_search(DataClass):
        dropout_rate = hp.Float(
            'dropout_rate',
            default=0.1,
            min_value=0,
            max_value=0.5,
            step=0.1,
        )

    lr = hp.Float(
        'optimazer__lerarning_rate',
        default=1e-4,
        min_value=1e-6,
        max_value=1e-3,
    )
    beta1 = hp.Float(
        'optimazer__beta1',
        default=0.9,
        min_value=0.5,
        max_value=1.1,
        step=0.1,
    )
    beta2 = hp.Float(
        'optimazer_beta1',
        default=0.999,
        min_value=0.5,
        max_value=1.1,
        sampling='log',
    )

    class Full_search(DataClass):
        cn = CN_search()
        an = AN_search()
        ff = FF_search()
        optimizer = keras.optimizers.Adam(
            name="Adam",
            learning_rate=lr,
            beta_1=beta1,
            beta_2=beta2,
        )

    return Full_search()

In [8]:
# Build model
model_name = 'search1'
log_dir = f'Temp/callbacks/{model_name}({datetime.datetime.now().strftime("%H-%M-%S--%d.%m")})'
def search_base_model(hp):
    hyper_pars_data = configure(hp)
    pars_data = DataClass( models.m_base.PARAMETRS)
    pars = pars_data.Info_expanded
    pars.update(hyper_pars_data.Info_expanded)

    model = models.m_base.build_model(**pars)
    return model
log_dir

'Temp/callbacks/search1(21-22-41--28.09)'

In [9]:
# Build tuner
tuner = keras_tuner.GridSearch(
    hypermodel=search_base_model,
    objective="loss",
    executions_per_trial=1,
    directory=log_dir,
    project_name='parametrs',

)

In [10]:
# Callbacks
class SaveCallback(keras.callbacks.Callback):

    def on_train_batch_end(self, batch, logs=None):
        # if not batch%50:
        input()
            # !cp -a "/content/drive/MyDrive/LOB/Temp/callbacks" "/content/drive/MyDrive/LOB/Save"


callbacks = [
    keras.callbacks.TensorBoard(
        log_dir= log_dir,
        histogram_freq=1,
        update_freq=1,
    ),
    # tf.keras.callbacks.ModelCheckpoint(
    #     f"{save_path}/checkPoints",
    #     monitor="val_loss",
    #     verbose=0,
    #     save_best_only=False,
    #     save_weights_only=True,
    #     mode="auto",
    #     save_freq=50,
    #     options=None,
    #     initial_value_threshold=None,
    # )
]
if IS_COLAB: callbacks +=[SaveCallback()]
callbacks

In [11]:
# Train
tuner.search(
    ds_train,
    validation_data=ds_val,
    epochs=20,
    callbacks=callbacks,
)



Search: Running Trial #1

Value             |Best Value So Far |Hyperparameter
4                 |4                 |cn_layers
True              |True              |share_weights
0.1               |0.1               |dropout_rate
0.0001            |0.0001            |optimazer__lerarning_rate
0.9               |0.9               |optimazer__beta1
0.999             |0.999             |optimazer_beta1

Epoch 1/20
 243/2037 [==>...........................] - ETA: 5:56 - loss: 2.1146 - sp_acc: 0.4409 - acc: 0.4782

KeyboardInterrupt: 